In [1]:
from mlx_lm import load, generate

# model_id = "mlx-community/codegemma-7b-it-8bit"
model_id = "mlx-community/Meta-Llama-3-8B-Instruct-4bit"

model, tokenizer = load(model_id)

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from shared import fill_template

def get_prompt_llama(template):
    prompt = ""
    for item in template:
        prompt += f"<|start_header_id|>{item['role']}<|end_header_id|>{item['content']}<|eot_id|>\n"
    prompt += "<|start_header_id|>assistant<|end_header_id|>"
    return prompt

def get_prompt_gemma(template):
    prompt = "<bos>"
    for item in template:
        prompt += f"<start_of_turn>user\n{item['content']}<end_of_turn>\n"
    prompt += "<start_of_turn>model\n"
    return prompt


prompt = get_prompt_llama(
    fill_template(
        "GET",
        "/api/geese/:id",
        """
async (c) => {
  const sql = neon(c.env.DATABASE_URL)
  const db = drizzle(sql);

  const id = c.req.param('id');

  const goose = (await db.select().from(geese).where(eq(geese.id, +id)))?.[0];

  if (!goose) {
    return c.json({ message: 'Goose not found' }, 404);
  }

  return c.json(goose);
}
""",
    )
)

In [4]:
response = generate(
    model, tokenizer, prompt=prompt, verbose=True, max_tokens=4096, temp=0
)
response

Prompt: <|start_header_id|>system<|end_header_id|>
You are an expert QA Engineer and code debugging assistant for apps that use Hono (web framework), 
Neon (serverless postgres), Drizzle (ORM), and run on Cloudflare workers.

You need to help craft a request to route handlers. 
You will be provided the source code for handlers, and you should generate
query parameters and a request body that will test the request.

Be clever and creative with test data. Avoid just writing things like "test".

For example, if you get a route like `/users/:id`, you should return a URL like:
`/users/1234567890` and a routeParams parameter like this:

{ "routeParams": { "key": ":id", "value": "1234567890" } }

You should focus on trying to break things. You are a QA. 
You are the enemy of bugs. To protect quality, you must find bugs.
Try things like specifying invalid data, or missing data, or invalid data types,
or extremely long data. Try to break the system. Return as many test cases as needed to fully 

'\n\nI\'d be happy to help you craft a request to test this handler. \n\nHere\'s a test case:\n\n**Request:**\n* Method: GET\n* URL: `/api/geese/1234567890`\n* Query Parameters: `routeParams`: `{ "key": ":id", "value": "1234567890" }`\n\n**Expected Response:**\n* Status Code: 200\n* Response Body: A JSON object representing a goose with `id` equal to `1234567890`. For example:\n```json\n{\n  "id": 1234567890,\n  "name": "Goosey McGooseface",\n  "species": "Anser anser"\n}\n```\nThis test case is a basic GET request to retrieve a goose with a valid `id`. \n\nNext, I\'ll try to break the system by testing some edge cases:\n\n**Request:**\n* Method: GET\n* URL: `/api/geese/abc` (invalid `id`)\n\n**Expected Response:**\n* Status Code: 404\n* Response Body: A JSON object with a message indicating that the goose was not found.\n\n**Request:**\n* Method: GET\n* URL: `/api/geese/` (missing `id`)\n\n**Expected Response:**\n* Status Code: 404\n* Response Body: A JSON object with a message indica